# Optimization

In this tutorial, we will see how gradient descent optimizes a model with a simple example in Pytorch.

More examples in TensorFlow: https://github.com/Jaewan-Yun/optimizer-visualization

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"       

DEVICE = 'cpu' #'cuda:0'

REAL_PARAMS = [1.2, 2.5]

FIG_NUM = 0


In [ ]:
% matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

plt.ion()

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

device = torch.device(DEVICE if torch.cuda.is_available() else "cpu")

## A simple example

The model we need to fit is $$ y=\sin \left( b \cos\left(ax\right) \right). $$
Here, $a$ and $b$ are the parameters to optimize.

The criterion for computing cost is mean squared error function.

In [ ]:
num_samples = 200
X = torch.linspace(-1, 1, num_samples, device=device)
f = lambda x, a, b: torch.sin(b * torch.cos(a * x))
noise = torch.randn(num_samples, device=device) / 10
y = f(X, *REAL_PARAMS) + noise

criterion = nn.MSELoss()


We visualize the 3D loss landscape for all samples as follows.

In [ ]:
FIG_NUM += 1
fig = plt.figure(FIG_NUM, figsize=(8, 6))
ax = fig.gca(projection='3d')
a3D, b3D = np.meshgrid(np.linspace(-3, 10, 100), np.linspace(-3, 10, 100))  # parameter space
cost3D = np.array([criterion(f(X, float(a_), float(b_)), y).detach().cpu().numpy()
                   for a_, b_ in zip(a3D.flatten(), b3D.flatten())]).reshape(a3D.shape)
ax.plot_surface(a3D, b3D, cost3D, rstride=1, cstride=1, cmap='jet', alpha=0.7)
plt.show();

In [ ]:
from collections import defaultdict
from statistics import mean

# define the levels to show in contour ploting
CONTOUR_LEVELS = np.concatenate((np.linspace(0, 0.1, 5, endpoint=False),
                                 np.linspace(0.1, 1, 4, endpoint=False),
                                 np.linspace(1, 2, 5, endpoint=False),
                                 np.linspace(2, 4, 5)))


def train(init_params, epochs, lr, batch_size, weight_decay, sgdm_momentum, lr_lambda):
    
    # training data
    dataloader = torch.utils.data.DataLoader(list(zip(X, y)), batch_size=batch_size, shuffle=True)

    # optimizers
    get_optimizer_dict = {
        'sgd': lambda p: optim.SGD(p, lr=lr, weight_decay=weight_decay),
        'sgdm': lambda p: optim.SGD(p, lr=lr, momentum=sgdm_momentum, weight_decay=weight_decay),
        'adgrad': lambda p: optim.Adagrad(p, lr=lr, weight_decay=weight_decay),
        'rmsprop': lambda p: optim.RMSprop(p, lr=lr, weight_decay=weight_decay),
        'adam': lambda p: optim.Adam(p, lr=lr, weight_decay=weight_decay),
        'adamax': lambda p: optim.Adamax(p, lr=lr, weight_decay=weight_decay),
    }
    
    # recorders of parameters and cost
    a_list, b_list, cost_list = defaultdict(list), defaultdict(list), defaultdict(list) 

    for name, get_optimizer in get_optimizer_dict.items():
        
        # initialize parameters
        a, b = [torch.tensor(float(p), device=device, requires_grad=True) for p in init_params]
        
        # get an optimizer
        optimizer = get_optimizer((a, b))
        
        # scheduler of changing the learning rate by epoch
        scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

        for epoch in range(epochs):
            scheduler.step()   # change the learning rate
                        
            for input, target in dataloader:
                # record parameters
                a_list[name].append(float(a))
                b_list[name].append(float(b))    
                
                optimizer.zero_grad()

                output = f(input, a, b)
                loss = criterion(output, target)
                
                # record cost
                cost_list[name].append(float(loss))

                loss.backward()
                optimizer.step()

    return a_list, b_list, cost_list

def plot_fit_functions(a_list, b_list):
    global FIG_NUM

    FIG_NUM +=1
    plt.figure(FIG_NUM, figsize=(5, 4))
    
    plt.scatter(X.cpu().numpy(), y.cpu().numpy(), s=5)    # training data

    lines = []
    for name in a_list.keys():
        pred = f(X, a_list[name][-1], b_list[name][-1]).detach().cpu().numpy()       # prediction
        line, = plt.plot(X.cpu().numpy(), pred, '-', lw=1, alpha=.7)                 # fit curve          
        lines.append(line)

    plt.legend(lines, a_list.keys())

def plot_optimization_steps_2d(a_list, b_list, cost_list):
    global FIG_NUM

    FIG_NUM +=1
    plt.figure(FIG_NUM, figsize=(8, 6))
    
    CS = plt.contour(a3D, b3D, cost3D, CONTOUR_LEVELS, cmap='jet')
    plt.clabel(CS, inline=1)

    plt.scatter(a_list['sgd'][0], b_list['sgd'][0],  s=30)    # initial parameter place
    plt.xlabel('a'); 
    plt.ylabel('b')

    lines = []
    for name in a_list.keys(): 
        line, = plt.plot(a_list[name], b_list[name], lw=1, alpha=.7)    # plot 2d gradient descent
        lines.append(line)

    plt.legend(lines, a_list.keys())

def plot_optimization_steps_3d(a_list, b_list, cost_list):
    global FIG_NUM

    FIG_NUM +=1
    fig = plt.figure(FIG_NUM, figsize=(8, 6))
    ax = fig.gca(projection='3d')

    ax.scatter(a_list['sgd'][0], b_list['sgd'][0], zs=cost_list['sgd'][0], s=50, c='r')  # initial parameter place
    ax.set_xlabel('a')
    ax.set_ylabel('b')
    
    ax.plot_surface(a3D, b3D, cost3D, rstride=1, cstride=1, cmap='jet', alpha=0.7)


    lines = []
    for name in a_list.keys(): 
        line, = ax.plot(a_list[name], b_list[name], zs=cost_list[name], zdir='z', lw=1)    # plot 3D gradient descent
        lines.append(line)

    plt.legend(lines, a_list.keys())
    plt.show()
    
def do_experiments(init_params=(2., 4.5), epochs=50, lr=0.01, batch_size=16, weight_decay=0, sgdm_momentum=0.9,
                   lr_lambda=lambda epoch: 1):
    lists = train(init_params, epochs, lr, batch_size, weight_decay, sgdm_momentum, lr_lambda)
    plot_fit_functions(*lists[:2])
    plot_optimization_steps_2d(*lists)
    
    return lists

### Experiment 1

Initial parameters: $a = 2$, $b = 4.5$

Epochs: 50

Learning rate: 0.01

Batch size: 16

Coefficient of L2 regularization (weight decay): 0

In [ ]:
lists = do_experiments()

In [ ]:
plot_optimization_steps_3d(*lists)

### Experiment 2

Initial parameters: $a = 2$, $b = 4.5$

Epochs: 50

Learning rate: 0.01

Batch size: 16

Coefficient of L2 regularization (weight decay): 0.01

In [ ]:
do_experiments(weight_decay=0.01);

### Experiment 3

Initial parameters: $a = 2$, $b = 4.5$

Epochs: 200

Learning rate: $0.01 \times 0.9^{epoch}$

Batch size: 16

Coefficient of L2 regularization (weight decay): 0

In [ ]:
do_experiments(epochs=200, lr_lambda=lambda epoch: 0.9**epoch);

### Experiment 4

Initial parameters: $a = 2$, $b = 4.5$

Epochs: 200

Learning rate: $0.02 \times 0.9^{epoch}$

Batch size: 16

Coefficient of L2 regularization (weight decay): 0

In [ ]:
do_experiments(lr=0.02, weight_decay=0.001, epochs=200, lr_lambda=lambda epoch: 0.9**epoch);

### Experiment 5

Initial parameters: $a = 5$, $b = 3$

Epochs: 200

Learning rate: $0.1 \times 0.9^{epoch}$

Batch size: 16

Coefficient of L2 regularization (weight decay): $1\times10^{-4}$

In [ ]:
do_experiments((5., 3.), lr=0.1, epochs=200, weight_decay=0.0001,
               lr_lambda=lambda epoch: 0.9**epoch);

## Recommendations for parameter initialization

### Xavier: Tanh, Sigmoid
Xavier samples values from $\mathcal{N}(0, \sigma)$.

$$\sigma = \text{gain} \times \sqrt{\frac{2}{\text{#in} + \text{#out}}}$$

### Kaiming: ReLU, Leaky ReLU

Kaiming samples values from $\mathcal{N}(0, \sigma)$

$$\sigma = \sqrt{\frac{2}{(1 + a^2) \times \text{#in}}}$$

Here, $a$ is the negative slope for Leaky ReLU (0 for ReLU).


In [ ]:
module = nn.Linear(5, 5)
print(module.weight)
nn.init.kaiming_normal_(module.weight, nonlinearity='relu')